In [17]:
exec(open(f"/Users/yerik/_apple_source/PY/GLOBAL/_1_paths.py",encoding="utf-8").read()) #paths
direc= curr_direc ;# hi_jupy() 
# curr_direc

ATTN - > TO start coding in this NOTEBOOK & following FNS :::  run ::: hi_jupy()
ATTN at the end RUN > py and cleans py ready to be copied >  runpy()
ATTN - > TO get all GLOBAL, libs and functions & ALL_DATA inputs, stats, temp, tables files info > db_paths()
ATTN - > TO move files from input to curr_direc > move_files(input_folder_path, curr_direc)
ATTN - > TO COPY program > copy_files_and_subdirectories(path_2_copy, dest_direc) --- respects original copied
ATTN - > TO get all GAME PLANS TO EXPORT ALPHA and BETA programs  > game_plan()
define program_folder_name before running runpy()


In [23]:
from pydub import AudioSegment
from pydub.utils import mediainfo

### importing and converting the audio file to WAV format would be to perform an initial analysis of the audio file.

In [24]:
from pydub import AudioSegment
from pydub.utils import mediainfo
exec(open(f"{direc}/functions.py",encoding="utf-8").read()) #paths


#f'{direc}/{desk_path}/input/ms_podcast/vr.wav'

#  VARIABLES

In [27]:
input_noise_reduction = 1.0 
#(1.0)   Default, no extra reduction),
#1.5)  # More aggressive reduction
# 0.8)  # Less aggressive reduction

input_music_file = f'{desk_path}/input/ms_podcast/vr.wav'#convert_to_wav("input.mp3", "output.wav")
output_wav =f'{desk_path}/output/ms_podcast/tostem.wav'
baseline_wav =output_wav 


# preprocessing

In [28]:
#convert_to_wav("input.mp3", "output.wav")

convert_to_wav(input_music_file, output_wav)

File has been converted and saved at /Users/yerik/Desktop/output/ms_podcast/tostem.wav


# 1 EXTREACT STEMS and TAKE VOCALS as/to 


!python -m demucs.separate -d cpu "/Users/yerik/Desktop/output/ms_podcast/vr_WAV_.wav" -o "/Users/yerik/Desktop/output/ms_podcast/STEMS"


In [6]:
import shutil

# ^^^ path ^^^
source_path = f"{desk_path}/output/ms_podcast/STEMS/htdemucs/tostem/vocals.wav"
# ^^^ path ^^^
destination_path = f"{desk_path}/input/ms_podcast/1_vr_wav_vocal.wav"

# Copy the file
shutil.copy2(source_path, destination_path)


'/Users/yerik/Desktop/input/ms_podcast/1_vr_wav_vocal.wav'

## 2.0  >>  Calculate the length of the audio in seconds and minutes.
## 2.1 >> Determine the number of channels (Mono/Stereo).
## 2.2 >> Retrieve the frame rate and other properties.

In [7]:

length, channels, frame_rate = analyze_audio('/Users/yerik/Desktop/input/ms_podcast/1_vr_wav_vocal.wav')  # ^^^ path ^^^
print('\n\n',length, channels, frame_rate)

Audio duration: 165.900952 seconds
Channels: 2
Frame rate: 44100 Hz


 165.900952 2 44100


## 3 >> NOISE REDUTCTION (basic and adjustable)

In [8]:
from scipy.io import wavfile
import numpy as np

In [9]:
# BASIC 
input_path ='/Users/yerik/Desktop/input/ms_podcast/1_vr_wav_vocal.wav'
output_path = f'{desk_path}/input/ms_podcast/2_vr_noise_reduction.wav'

basic_noise_reduction(input_path, output_path)


Noise-reduced audio saved at /Users/yerik/Desktop/input/ms_podcast/2_vr_noise_reduction.wav


In [10]:
# # ADJUSTABLE 
# adjustable_noise_reduction(input_path, output_path, threshold_factor=input_noise_reduction)  # Less aggressive reduction
# print(f'Your Choice was {input_noise_reduction}, \n\n')

Explanation
butter_highpass: Creates a high-pass filter using Butterworth design parameters.
Input: cutoff frequency, sampling rate, order of the filter.
Output: Filter coefficients a and b.
highpass_filter: Applies the high-pass filter to audio data.
Input: audio data, cutoff frequency, sampling rate, order of the filter, and threshold factor for aggressive filtering.
Output: Filtered audio data.
adjustable_noise_reduction: A wrapper function that allows users to adjust the noise reduction parameters.
Input: input_path for the input .wav file, output_path for the output .wav file, and threshold_factor for more or less aggressive noise reduction.
Output: Writes the filtered data into a .wav file.

# Hight PASS

In [5]:
from scipy.io import wavfile
from scipy.signal import butter, filtfilt
import numpy as np

def adjustable_highpass_filter(input_path, output_path, cutoff_frequency, filter_order, threshold_factor):

    def butter_highpass(cutoff, fs, order=5):
        nyquist = 0.5 * fs
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='high', analog=False)
        return b, a

    def highpass_filter(data, cutoff, fs, order=5, threshold_factor=1):
        b, a = butter_highpass(cutoff, fs, order=order)
        y = filtfilt(b, a, data)
        y = np.where(np.abs(y) > threshold_factor * np.max(np.abs(y)), y, 0)
        return y.astype(np.int16)

    # Read the input audio file
    fs, data = wavfile.read(input_path)  # ^^^ path ^^^
    
    # Check if the audio is stereo
    if len(data.shape) == 2:
        channel1 = data[:, 0]
        channel2 = data[:, 1]
        
        # Apply high-pass filter to each channel
        try:
            filtered_channel1 = highpass_filter(channel1, cutoff_frequency, fs, filter_order, threshold_factor)
            filtered_channel2 = highpass_filter(channel2, cutoff_frequency, fs, filter_order, threshold_factor)
        except ValueError as e:
            print(f"Error: {e}")
            return

        # Merge channels back into a stereo audio file
        filtered_stereo = np.column_stack((filtered_channel1, filtered_channel2))

        # Write the filtered audio to a new file
        wavfile.write(output_path, fs, filtered_stereo)  # ^^^ path ^^^

    else:
        print("The audio file is not in stereo format.")


# Here, replace desk_path, input_noise_reduction, filter_order, and cutoff_frequency with your actual values.
input_path = f'{desk_path}/input/ms_podcast/2_vr_noise_reduction.wav'
output_path = f'{desk_path}/input/ms_podcast/3_vr_highpass.wav'
input_noise_reduction = 0.05
filter_order = 6
cutoff_frequency = 100.0

adjustable_highpass_filter(input_path, output_path, cutoff_frequency, filter_order, input_noise_reduction)


NameError: name 'desk_path' is not defined

# Preprocess sound in order to make it a video

In [7]:
import soundfile as sf
import numpy as np
import os

def cut_and_export_audio(input_file, num_pieces, output_folder):
    # ^^^ path ^^^
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Read audio file
    data, sample_rate = sf.read(input_file)

    # Calculate the length of each piece
    piece_length = len(data) // num_pieces

    # Split and export the audio
    for i in range(num_pieces):
        start = i * piece_length
        end = (i + 1) * piece_length
        output_file = os.path.join(output_folder, f"piece_{i+1}.wav")
        # ^^^ path ^^^
        sf.write(output_file, data[start:end], sample_rate)

# Set parameters
input_file = f"{desk_path}/input/ms_podcast/vr.wav"   # Input .wav file
num_pieces = 20  # Number of pieces to cut the audio into
output_folder = f"{desk_path}/input/ms_podcast/output_audio"  # Folder to store the cut pieces

# Execute function
cut_and_export_audio(input_file, num_pieces, output_folder)


# for every piece

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from moviepy.editor import VideoClip, AudioFileClip

# Function to create customized waveform video
def make_frame(t, audio_data, fs):
    start_idx = int(t * fs)
    end_idx = int((t + 0.1) * fs)
    audio_slice = audio_data[start_idx:end_idx]

    fig, ax = plt.subplots()
    fig.patch.set_facecolor('green')
    ax.set_facecolor('green')
    ax.plot(audio_slice, color='white')
    ax.set_ylim([-2**15, 2**15])
    ax.axis('off')

    corner_signs = '<<<'
    ax.text(0.05, 0.95, corner_signs, ha='left', va='top', transform=ax.transAxes, color='white')
    ax.text(0.95, 0.95, corner_signs[::-1], ha='right', va='top', transform=ax.transAxes, color='white')
    ax.text(0.05, 0.05, corner_signs, ha='left', va='bottom', transform=ax.transAxes, color='white')
    ax.text(0.95, 0.05, corner_signs[::-1], ha='right', va='bottom', transform=ax.transAxes, color='white')

    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)

    return frame

# Base paths for input and output # ^^^ path ^^^
desk_path = os.path.expanduser("~/Desktop")
num_pieces = num_pieces  # Adjust as needed

# Loop through each audio piece
for i in range(1, num_pieces+1):
    input_audio_path = f"{desk_path}/input/ms_podcast/output_audio/piece_{i}.wav"  # ^^^ path ^^^
    output_video_path = f"{desk_path}/input/ms_podcast/VID_FAST_piece_{i}.mp4"  # ^^^ path ^^^


    # Read the audio file
    fs, audio_data = wavfile.read(input_audio_path)

    # Create video without audio
    video = VideoClip(lambda t: make_frame(t, audio_data, fs), duration=(len(audio_data) / fs) - 0.1)
    
    # Add audio to the video
    audio = AudioFileClip(input_audio_path)
    video = video.set_audio(audio)
    
    # Render the video
    video.write_videofile(output_video_path, fps=10, audio_codec='aac')


Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_1.mp4.
MoviePy - Writing audio in VID_FAST_piece_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_1.mp4



Moviepy - Done !
Moviepy - video ready /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_1.mp4
Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_2.mp4.
MoviePy - Writing audio in VID_FAST_piece_2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_2.mp4



Moviepy - Done !
Moviepy - video ready /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_2.mp4
Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_3.mp4.
MoviePy - Writing audio in VID_FAST_piece_3TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_3.mp4



t:  95%|████████████████████████▌ | 1019/1078 [00:27<00:01, 37.84it/s, now=None]

In [13]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from moviepy.editor import VideoClip

# Function to create customized waveform video
def make_frame(t, audio_data, fs):
    start_idx = int(t * fs)
    end_idx = int((t + 0.1) * fs)
    audio_slice = audio_data[start_idx:end_idx]

    fig, ax = plt.subplots()
    fig.patch.set_facecolor('green')
    ax.set_facecolor('green')
    ax.plot(audio_slice, color='white')
    ax.set_ylim([-2**15, 2**15])
    ax.axis('off')

    corner_signs = '<<<'
    ax.text(0.05, 0.95, corner_signs, ha='left', va='top', transform=ax.transAxes, color='white')
    ax.text(0.95, 0.95, corner_signs[::-1], ha='right', va='top', transform=ax.transAxes, color='white')
    ax.text(0.05, 0.05, corner_signs, ha='left', va='bottom', transform=ax.transAxes, color='white')
    ax.text(0.95, 0.05, corner_signs[::-1], ha='right', va='bottom', transform=ax.transAxes, color='white')

    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)

    return frame

# ASCII Mushroom for branding
mushroom = """
(Your ASCII Mushroom Here)
"""

# Base paths for input and output # ^^^ path ^^^
desk_path = os.path.expanduser("~/Desktop")
num_pieces = 5  # Adjust as needed

# Loop through each audio piece
for i in range(1, num_pieces+1):
    input_audio_path = f"{desk_path}/input/ms_podcast/output_audio/piece_{i}.wav"  # ^^^ path ^^^
    output_video_path = f"{desk_path}/input/ms_podcast/VID_FAST_piece_{i}.mp4"  # ^^^ path ^^^

    # Read the audio file
    fs, audio_data = wavfile.read(input_audio_path)

    # Create video
    video = VideoClip(lambda t: make_frame(t, audio_data, fs), duration=(len(audio_data) / fs) - 0.1)
    video.write_videofile(output_video_path, fps=10)


                                                                                
t:   1%|▍                           | 30/2156 [01:27<01:14, 28.52it/s, now=None]
                                                                                
t:  68%|█████████████████▋        | 1462/2156 [00:43<00:15, 44.79it/s, now=None]

Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_1.mp4.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_1.mp4





t:   0%|                                     | 0/2156 [00:00<?, ?it/s, now=None]

t:   0%|                             | 6/2156 [00:00<00:42, 50.40it/s, now=None]

t:   1%|▏                           | 12/2156 [00:00<00:45, 47.27it/s, now=None]

t:   1%|▏                           | 17/2156 [00:00<00:46, 45.85it/s, now=None]

t:   1%|▎                           | 22/2156 [00:00<00:46, 45.73it/s, now=None]

t:   1%|▎                           | 27/2156 [00:00<00:46, 45.94it/s, now=None]

t:   1%|▍                           | 32/2156 [00:00<00:45, 46.59it/s, now=None]

t:   2%|▍                           | 37/2156 [00:00<00:46, 45.99it/s, now=None]

t:   2%|▌                           | 42/2156 [00:00<00:45, 46.18it/s, now=None]

t:   2%|▌                           | 47/2156 [00:01<00:45, 45.92it/s, now=None]

t:   2%|▋                           | 52/2156 [00:01<00:45, 46.17it/s, now=None]

t:   3%|▋                           | 57/2156 [00:01<00:45, 45.80it/s, now=None]

t:   3%|▊     

t:  21%|█████▋                     | 455/2156 [00:41<00:42, 40.09it/s, now=None]

t:  21%|█████▊                     | 460/2156 [00:41<00:42, 40.20it/s, now=None]

t:  22%|█████▊                     | 465/2156 [00:41<00:40, 42.07it/s, now=None]

t:  22%|█████▉                     | 470/2156 [00:41<00:39, 43.13it/s, now=None]

t:  22%|█████▉                     | 475/2156 [00:41<00:37, 44.33it/s, now=None]

t:  22%|██████                     | 480/2156 [00:41<00:38, 43.12it/s, now=None]

t:  22%|██████                     | 485/2156 [00:42<00:38, 43.91it/s, now=None]

t:  23%|██████▏                    | 490/2156 [00:42<00:37, 45.03it/s, now=None]

t:  23%|██████▏                    | 495/2156 [00:42<00:37, 43.80it/s, now=None]

t:  23%|██████▎                    | 500/2156 [00:42<01:17, 21.33it/s, now=None]

t:  23%|██████▎                    | 505/2156 [00:42<01:05, 25.18it/s, now=None]

t:  24%|██████▍                    | 510/2156 [00:43<00:56, 29.01it/s, now=None]

t:  24%|██████▍ 

t:  42%|███████████▎               | 899/2156 [00:53<00:45, 27.54it/s, now=None]

t:  42%|███████████▎               | 903/2156 [00:53<00:41, 30.03it/s, now=None]

t:  42%|███████████▎               | 907/2156 [00:53<00:39, 31.24it/s, now=None]

t:  42%|███████████▍               | 911/2156 [00:54<00:38, 32.25it/s, now=None]

t:  42%|███████████▍               | 915/2156 [00:54<00:39, 31.08it/s, now=None]

t:  43%|███████████▌               | 919/2156 [00:54<00:37, 32.87it/s, now=None]

t:  43%|███████████▌               | 923/2156 [00:54<00:37, 32.63it/s, now=None]

t:  43%|███████████▌               | 927/2156 [00:54<00:38, 31.64it/s, now=None]

t:  43%|███████████▋               | 931/2156 [00:54<00:38, 32.18it/s, now=None]

t:  43%|███████████▋               | 935/2156 [00:54<00:38, 31.97it/s, now=None]

t:  44%|███████████▊               | 939/2156 [00:54<00:36, 33.61it/s, now=None]

t:  44%|███████████▊               | 943/2156 [00:55<00:34, 35.04it/s, now=None]

t:  44%|████████

...

Moviepy - Done !
Moviepy - video ready /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_3.mp4



                                                                             
t:  68%|█████████████████▋        | 1462/2156 [04:42<00:15, 44.79it/s, now=None]
                                                                             
t:  68%|█████████████████▋        | 1462/2156 [04:42<00:15, 44.79it/s, now=None]

Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_4.mp4.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_4.mp4



t:  51%|█████████████▏            | 1091/2156 [00:43<12:14,  1.45it/s, now=None]

KeyboardInterrupt: 

In [ ]:
get_music_videos(desk_path, 10)

In [10]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from moviepy.editor import VideoClip

def get_music_videos(desk_path, num_pieces):
    # ASCII Mushroom
    mushroom = """
    ... (omitted for brevity)
    """

    # Function to create customized waveform video
    def make_frame(t, audio_data, fs):
        # Extract audio slice for the given time
        start_idx = int(t * fs)
        end_idx = int((t + 0.1) * fs)
        audio_slice = audio_data[start_idx:end_idx]

        # Create the figure and axis objects
        fig, ax = plt.subplots()

        # Customize your waveform video
        # ... (omitted for brevity)

        # Convert Matplotlib figure to RGB array
        fig.canvas.draw()
        frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))

        # Close the plot to save memory
        plt.close(fig)
        return frame

    # Loop through all the audio files you want to chop
    for i in range(1, num_pieces+1):
        input_audio_path = f"{desk_path}/input/ms_podcast/output_audio/piece_{i}.wav"  # ^^^ path ^^^
        output_video_path = f"{desk_path}/input/ms_podcast/VID_FAST_piece_{i}.mp4"  # ^^^ path ^^^

        # Read the audio file
        fs, audio_data = wavfile.read(input_audio_path)  # ^^^ path ^^^

        # Create video
        video = VideoClip(lambda t: make_frame(t, audio_data, fs), duration=(len(audio_data) / fs) - 0.1)
        video.write_videofile(output_video_path, fps=10)  # ^^^ path ^^^


In [11]:
get_music_videos(desk_path, 10)

Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_1.mp4.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_1.mp4



Moviepy - Done !
Moviepy - video ready /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_1.mp4
Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_2.mp4.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_2.mp4



Moviepy - Done !
Moviepy - video ready /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_2.mp4
Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_3.mp4.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_3.mp4



Moviepy - Done !
Moviepy - video ready /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_3.mp4
Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_4.mp4.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_piece_4.mp4



t:   1%|▍                           | 30/2156 [00:12<01:14, 28.52it/s, now=None]

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

# make a video with desired one 

In [7]:
input_audio_path = f"{desk_path}/input/ms_podcast/output_audio{1:num_pieces}"  #choose what is what
output_video_path = f"{desk_path}/input/ms_podcast/VID_FAST_1.mp4"

In [22]:
%%writefile /Users/yerik/Desktop/ms_tools/sets_video/run_3.py

exec(open(f"/Users/yerik/_apple_source/PY/GLOBAL/_1_paths.py",encoding="utf-8").read()) #paths

#Do a for loop to thsi following so it takes all the aaudio files from 1 to the one we decided to chop
input_audio_path = f"{desk_path}/input/ms_podcast/vr.wav"  #choose what is what
output_video_path = f"VID_FAST_1.mp4"


#background_video_path =  f"{desk_path}/
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from moviepy.editor import VideoClip
import imageio

# ^^^ path ^^^
background_image_path =  f"{desk_path}/input/ms_podcast/moon.jpeg"
# Or, if you're using a frame from a video as a background
# ^^^ path ^^^
#background_video_path =  f"{desk_path}

bg_image = imageio.imread(background_image_path)

def make_frame(t):
    start_idx = int(t * fs)
    end_idx = int((t + 0.1) * fs)
    audio_slice = audio_data[start_idx:end_idx]

    fig, ax = plt.subplots()

    ax.set_ylim([-2**15, 2**15])
    ax.set_xlim([0, len(audio_slice)])

    ax.imshow(bg_image, aspect='auto', extent=[0, len(audio_slice), -2**15, 2**15])

    ax.plot(audio_slice, color='white')
    ax.axis('off')

    # Additional decorations here...

    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)
    return frame

fs, audio_data = wavfile.read(input_audio_path)
video = VideoClip(make_frame, duration=(len(audio_data) / fs) - 0.1)
video.write_videofile(output_video_path, fps=10)

# # Import Libraries
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.io import wavfile
# from moviepy.editor import VideoClip

# def get_music_videos(input_audio_path):
#     # ASCII Mushroom
#     mushroom = """
#     ______________________________________________________________________
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__
#     _|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|
#     ___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|___|__

#     """

#     # Function to create customized waveform video
#     def make_frame(t):
#         start_idx = int(t * fs)
#         end_idx = int((t + 0.1) * fs)  # Smaller slice for faster movement

#         audio_slice = audio_data[start_idx:end_idx]

#         # Create the figure and axis objects
#         fig, ax = plt.subplots()

#         # Set green background
#         fig.patch.set_facecolor('green')
#         ax.set_facecolor('green')

#         # Plotting the waveform, make it span almost the entire space
#         ax.plot(audio_slice, color='white')
#         ax.set_ylim([-2**15, 2**15])

#         # Remove the axes
#         ax.axis('off')

#         # Add your DJ name to each corner
#         plt.text(0.05, 0.95, '<<<', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes, color='white')
#         plt.text(0.95, 0.95, '>>>', horizontalalignment='right', verticalalignment='top', transform=ax.transAxes, color='white')
#         plt.text(0.05, 0.05, '<<<', horizontalalignment='left', verticalalignment='bottom', transform=ax.transAxes, color='white')
#         plt.text(0.95, 0.05, '>>>', horizontalalignment='right', verticalalignment='bottom', transform=ax.transAxes, color='white')

#         # Add ASCII Mushroom at the bottom
#         plt.text(0.5, 0.01, mushroom, horizontalalignment='center', verticalalignment='bottom', transform=ax.transAxes, fontsize=8, color='white')

#         # Convert Matplotlib figure to RGB array
#         fig.canvas.draw()
#         frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
#         frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))

#         # Close the plot to save memory
#         plt.close(fig)

#         return frame

#     # Read the audio file
#     fs, audio_data = wavfile.read(input_audio_path)

#     # Create video
#     video = VideoClip(make_frame, duration=(len(audio_data) / fs) - 0.1)  # Smaller duration for faster movement
#     video.write_videofile(output_video_path, fps=10)  # Higher fps for faster movement


Overwriting /Users/yerik/Desktop/ms_tools/sets_video/run_3.py


In [46]:
# backgroun image 

In [19]:
input_audio_path = f"{desk_path}/input/ms_podcast/2_vr_noise_reduction.wav"  #choose what is what
output_video_path = f"{desk_path}/input/ms_podcast/VID_FAST_2.mp4"

In [20]:
#background_video_path =  f"{desk_path}/
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from moviepy.editor import VideoClip
import imageio

# ^^^ path ^^^
background_image_path =  f"{desk_path}/input/ms_podcast/moon.jpeg"
# Or, if you're using a frame from a video as a background
# ^^^ path ^^^
#background_video_path =  f"{desk_path}

bg_image = imageio.imread(background_image_path)

def make_frame(t):
    start_idx = int(t * fs)
    end_idx = int((t + 0.1) * fs)
    audio_slice = audio_data[start_idx:end_idx]

    fig, ax = plt.subplots()

    ax.set_ylim([-2**15, 2**15])
    ax.set_xlim([0, len(audio_slice)])

    ax.imshow(bg_image, aspect='auto', extent=[0, len(audio_slice), -2**15, 2**15])

    ax.plot(audio_slice, color='white')
    ax.axis('off')

    # Additional decorations here...

    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)
    return frame

fs, audio_data = wavfile.read(input_audio_path)
video = VideoClip(make_frame, duration=(len(audio_data) / fs) - 0.1)
video.write_videofile(output_video_path, fps=10)


Moviepy - Building video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_2.mp4.
Moviepy - Writing video /Users/yerik/Desktop/input/ms_podcast/VID_FAST_2.mp4



Moviepy - Done !
Moviepy - video ready /Users/yerik/Desktop/input/ms_podcast/VID_FAST_2.mp4


# sound to vid

In [52]:
from moviepy.editor import VideoFileClip, AudioFileClip

# ^^^ path ^^^
video_path = f"{desk_path}/input/ms_podcast/VID_FAST_2.mp4" # Replace with the path to your video file
audio_path = f"{desk_path}/input/ms_podcast/2_vr_noise_reduction.wav" # Replace with the path to your audio file

# Load video and audio files
video = VideoFileClip(video_path)
audio = AudioFileClip(audio_path)

# Set audio to the video
video = video.set_audio(audio)

# ^^^ path ^^^
output_path = f"{desk_path}/output/ms_podcast/vr_out_music_video.mp4"  # Replace with the path where you want to save the output video

# Write the result to a file
video.write_videofile(output_path, audio_codec='aac')


Moviepy - Building video /Users/yerik/Desktop/output/ms_podcast/vr_out_music_video.mp4.
MoviePy - Writing audio in vr_out_music_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /Users/yerik/Desktop/output/ms_podcast/vr_out_music_video.mp4



Moviepy - Done !
Moviepy - video ready /Users/yerik/Desktop/output/ms_podcast/vr_out_music_video.mp4


In [13]:
# from scipy.io import wavfile
# import numpy as np
# import scipy.fftpack

# def advanced_noise_reduction(input_path, output_path):
#     rate, data = wavfile.read(input_path)
#     w = np.fft.fft(data)
#     freqs = np.fft.fftfreq(len(w))
    
#     # Assuming that noise is concentrated at lower frequencies, we apply a high-pass filter
#     # Change these values based on your specific needs
#     min_freq = 500  # in Hz
#     max_freq = 3000  # in Hz
    
#     # Zero out components that are not within the desired frequency range
#     w[(freqs < min_freq)] = 0
#     w[(freqs > max_freq)] = 0
    
#     # Inverse FFT to get the audio back
#     filtered_data = np.fft.ifft(w).real.astype(np.int16)
    
#     # Save the noise-reduced audio
#     wavfile.write(output_path, rate, filtered_data)
#     print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# # Example desk_path for demonstration purposes; replace it with your actual desk_path
# #desk_path = "/your/desk/path"

# # Example usage
# input_path = f'{desk_path}/input/ms_podcast/vr.wav'
# output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced_2.wav'
# advanced_noise_reduction(input_path, output_path)


In [14]:
# from scipy.io import wavfile
# import numpy as np
# import scipy.fftpack

# def advanced_noise_reduction(input_path, output_path):
#     rate, data = wavfile.read(input_path)
#     w = np.fft.fft(data)
#     freqs = np.fft.fftfreq(len(w))
    
#     # Apply a less aggressive high-pass filter
#     min_freq = 100  # in Hz
    
#     # Zero out components that are not within the desired frequency range
#     w[(freqs < min_freq)] = 0
    
#     # Inverse FFT to get the audio back
#     filtered_data = np.fft.ifft(w).real.astype(np.int16)
    
#     # Save the noise-reduced audio
#     wavfile.write(output_path, rate, filtered_data)
#     print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# # Example desk_path for demonstration purposes; replace it with your actual desk_path
# #desk_path = "/your/desk/path"

# # Example usage
# input_path = f'{desk_path}/input/ms_podcast/vr.wav'
# output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced_2.wav'
# advanced_noise_reduction(input_path, output_path)


# high pass

In [16]:
# import librosa
# import scipy.signal
# from scipy.io.wavfile import write

# def simple_highpass_filter(input_path, output_path):
#     # Load audio file
#     y, sr = librosa.load(input_path, sr=None)
    
#     # Create a high-pass filter
#     sos = scipy.signal.butter(10, 100, 'hp', fs=sr, output='sos')
    
#     # Apply filter
#     filtered_signal = scipy.signal.sosfilt(sos, y)
    
#     # Save the noise-reduced audio
#     write(output_path, sr, filtered_signal.astype(np.int16))
#     print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# # Example desk_path for demonstration purposes; replace it with your actual desk_path
# #desk_path = "/your/desk/path"

# # Example usage
# input_path = f'{desk_path}/input/ms_podcast/vr.wav'
# output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced_3.wav'
# simple_highpass_filter(input_path, output_path)


Noise-reduced audio saved at /Users/yerik/Desktop/output/ms_podcast/vr_noise_reduced_3.wav


In [17]:
import librosa
import numpy as np
from scipy.io.wavfile import write

def minimal_processing(input_path, output_path):
    # Load audio file
    y, sr = librosa.load(input_path, sr=None)
    
    # Remove silence
    yt, _ = librosa.effects.trim(y)
    
    # Normalize audio
    yt = librosa.util.normalize(yt)
    
    # Save the audio
    write(output_path, sr, (yt * 32767).astype(np.int16))
    print(f"Processed audio saved at {output_path}")  # ^^^ path ^^^

# Example desk_path for demonstration purposes; replace it with your actual desk_path
#desk_path = "/your/desk/path"

# Example usage
input_path = f'{desk_path}/input/ms_podcast/vr.wav'
output_path = f'{desk_path}/output/ms_podcast/vr_minimal_processed.wav'
minimal_processing(input_path, output_path)


Processed audio saved at /Users/yerik/Desktop/output/ms_podcast/vr_minimal_processed.wav


In [ ]:
import librosa
import numpy as np
from scipy.io.wavfile import write

def spectral_subtraction(input_path, output_path, alpha=1.0):
    # Load audio file
    y, sr = librosa.load(input_path, sr=None)
    
    # Compute the short-time Fourier transform (STFT)
    D = librosa.stft(y)
    
    # Estimate the noise
    noise_est = np.median(np.abs(D), axis=1)
    
    # Perform spectral subtraction
    D_denoised = D - alpha * noise_est[:, None]
    
    # Ensure no negative values
    D_denoised = np.maximum(D_denoised, 0.0)
    
    # Inverse STFT to convert back to time domain
    y_denoised = librosa.istft(D_denoised)
    
    # Save the audio
    write(output_path, sr, (y_denoised * 32767).astype(np.int16))
    print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# Example desk_path for demonstration purposes; replace it with your actual desk_path
desk_path = "/your/desk/path"

# Example usage
input_path = f'{desk_path}/input/ms_podcast/vr.wav'
output_path = f'{desk_path}/output/ms_podcast/vr_noise_reduced.wav'
spectral_subtraction(input_path, output_path, alpha=2.0)


# FUNCTIONS

In [10]:
%%writefile functions.py

def convert_to_wav(input_path, output_path):
    # Supported audio formats by pydub
    SUPPORTED_FORMATS = ["mp3", "flv", "ogg", "mp4", "wav", "aif", "wma", "aac", "m4a"]

    # Extract the file extension from input_path
    file_extension = input_path.split('.')[-1].lower()

    # Check if the file format is supported
    if file_extension not in SUPPORTED_FORMATS:
        print(f"Unsupported file format: {file_extension}")
        return

    # Read the audio file
    audio = AudioSegment.from_file(input_path, format=file_extension)

    # Convert to WAV
    audio.export(output_path, format="wav")
    print(f"File has been converted and saved at {output_path}")  # ^^^ path ^^^


# ------------------------------------------------------------------- Function to analyze audio
def analyze_audio(audio_path):
    #from pydub.utils import mediainfo
    audio_info = mediainfo(audio_path)
    length_in_seconds = float(audio_info["duration"])
    channels = int(audio_info["channels"])
    frame_rate = int(audio_info["sample_rate"])

    print(f"Audio duration: {length_in_seconds} seconds")
    print(f"Channels: {channels}")
    print(f"Frame rate: {frame_rate} Hz")
    
    return length_in_seconds, channels, frame_rate


## ------------------------------------------------------------------- Noise Reduction 

def basic_noise_reduction(input_path, output_path):
    # Read the input audio file
    rate, data = wavfile.read(input_path)
    
    # Take a sample of the noise (adjust as needed)
    noise_sample = data[:5000]
    
    # Generate noise profile
    noise_profile = np.mean(np.abs(noise_sample))
    
    # Perform noise reduction: Zero out anything below the profile
    reduced_noise_data = np.where(np.abs(data) < noise_profile, 0, data)
    
    # Save the noise-reduced audio
    wavfile.write(output_path, rate, reduced_noise_data.astype(np.int16))
    print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

# ## ------------------------------------------------------------------- ADJUSTABLE Noise Reduction 


def adjustable_noise_reduction(input_path, output_path, threshold_factor=1.0):
    # Read the input audio file
    rate, data = wavfile.read(input_path)
    
    # Take a sample of the noise (adjust the range as needed)
    noise_sample = data[:5000]
    
    # Generate noise profile
    noise_profile = np.mean(np.abs(noise_sample))
    
    # Calculate the adjusted threshold
    adjusted_threshold = noise_profile * threshold_factor
    
    # Perform noise reduction: Zero out anything below the adjusted threshold
    reduced_noise_data = np.where(np.abs(data) < adjusted_threshold, 0, data)
    
    # Save the noise-reduced audio
    wavfile.write(output_path, rate, reduced_noise_data.astype(np.int16))
    print(f"Noise-reduced audio saved at {output_path}")  # ^^^ path ^^^

Overwriting functions.py
